In [1]:
# Prioritization of Securities for Analysis

In [2]:
# Modules used in this notebook

from IPython.display import display

import numpy as np
import pandas as pd

from datetime import datetime, date

import glob
import os

# Quantitative Finance - Option Valuation library
import QuantLib as ql

In [3]:
# Constants and Global Variables

# Group of Securities
# [TODO:] Need a better system to manage securities included for analysis.
#  Manually copying the list is not efficient.

# portfolio holdings
etf_holdings = ['TQQQ', 'UPRO']
stock_holdings = ['META', 'AAPL', 'PLTR', 'GPRO', 'C']

# trades executed with year
etf_trades = ['TLT', 'SOXL', 'SQQQ', 'DIA', 'USO']
stock_trades = ['ABNB', 'ADP', 'AEO', 'AMC', 'AMZN', 'AXP', 'BAC', 'BIIB', 'BTU', 'BX', \
                'CGC', 'CHPT', 'CI', 'CLF', 'CLOV', 'CMCSA', 'COST', 'CRM', 'DAL', 'DE', \
                'DIS', 'DOCU', 'DOW', 'DVN', 'GME', 'GOOGL', 'HD', 'IBM', 'JNJ', 'KR', \
                'LMT', 'M', 'MRK', 'MSFT', 'MU', 'NFLX', 'NIO', 'NKE', 'NVDA', 'OKTA', \
                'PANW', 'PEP', 'PFE', 'PINS', 'PYPL', 'RAD', 'RBLX', 'RKT', 'SE', 'SNAP', \
                'SNOW', 'SOFI', 'TSLA', 'TSM', 'UAL', 'UNH', 'UPS', 'VZ', 'WFC', 'WMT', \
                'XOM', 'ZM']

# index etfs for tracking
index_etfs = ['^SPX', '^VIX', 'SPY', 'QQQ', 'IWM', 'LQD', 'UVXY', 'VXX']

# new etfs and stocks under consideration
new_etfs = ['XBI', 'XLK', 'XLY', 'XLF', 'XLE', 'XOP', 'ARKK', 'TBT', 'ARKG', 'SMH']
new_stocks = []

# potential earnings play
earning_stocks = []

# etf_list is used to filter earnings report from TipRanks
etf_list = etf_holdings + etf_trades + index_etfs + new_etfs

master_list = etf_holdings + stock_holdings + etf_trades + stock_trades + index_etfs + new_stocks + new_etfs + earning_stocks

# remove duplicates while keeping order
securities = sorted(set(master_list), key=master_list.index)

# debug
#securities = set(['CRM', 'SPLK', 'SNOW', 'OKTA', 'KR'])
#securities = set(etf_holdings + stock_holdings)
#securities = set([ticker for ticker in earning_stocks if ticker not in master_list])

In [4]:
#print(sorted(set(new_earning_stocks)))

In [5]:
# Functions used

# Import option data for security

def get_option_data(security):
    # Input:
    #  security - Ticker symbol for security of interest
    # Return:
    #  dataframe of option data
    data_type = 'options'

    # get latest data file for security
    file_name = max(glob.iglob('data/{}_{}_*.csv'.format(security, data_type)), key=os.path.getctime)

    return pd.read_csv(file_name, index_col=None, parse_dates=True, infer_datetime_format=True)

def get_calendar_data(security):
    # Get Corporate Activity Calendar
    try:
        file_name = max(glob.iglob('data/{}_calendar.csv'.format(security)), key=os.path.getctime)
        calendar_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
        calendar_df = calendar_df.T
        calendar_df['Earnings Date'] = pd.to_datetime(calendar_df['Earnings Date'], infer_datetime_format=True, utc=True)
        calendar_df.set_index('Earnings Date', inplace=True)
        # change data type from string to numeric
        calendar_df = calendar_df.apply(pd.to_numeric)
    except:
        calendar_df = None

    return calendar_df

def get_earnings_data(security):
    # Get Earnings History
    try:
        file_name = 'data/{}_earnings_hist.csv'.format(security)
        earnings_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
        earnings_df.index = pd.to_datetime(earnings_df.index, utc=True, infer_datetime_format=True)
        earnings_df.index = earnings_df.index.tz_convert('America/New_York')
        earnings_df.dropna(axis=0, how='all', inplace=True)
        earnings_df = earnings_df.head(1)
    except:
        earnings_df = None
        
    return earnings_df

def get_tipranks_earnings_data(security):
    try:
        file_name = 'data/{}_earnings_hist_tipranks.csv'.format(security)
        tipranks_earnings_df = pd.read_csv(file_name, index_col=0, parse_dates=True, infer_datetime_format=True)
        tipranks_earnings_df = tipranks_earnings_df.head(1)
    except:
        tipranks_earnings_df = None
        
    return tipranks_earnings_df

In [6]:
# Initialize for results
result_dict = dict()
calendar_result_dict = dict()
tipranks_result_dict = dict()
security_low_price_set = set()
security_set = set()
security_low_set = set()

# Number of stocks to display for each criteria
count = 5

# Only consider stock with price above threshold
stock_price_threshold = 7.00
# Only consider stock with option open interest above threshold
open_interest_threshold = 100000

# lotto dicts
lotto_call_dict = dict()
lotto_put_dict = dict()

# calendar spread
cs_call_dict = dict()

In [7]:
# Import data for all securities
for security in securities:
    print(security)
    # Read option data from latest stored CSV file
    option_df = get_option_data(security)
            
    # Read corporate activity calendar data from latest stored CSV file
    calendar_df = get_calendar_data(security)
    
    # Read Earninhs history data
    earnings_df = get_earnings_data(security)
    
    # Read Tipranks Earnings data
    tipranks_earnings_df = get_tipranks_earnings_data(security)

    # Relevant info extract from corporate activity calendar
    if calendar_df is not None:
        earnings_disperson = 100.0 * (calendar_df['Earnings High'] - calendar_df['Earnings Low']) / calendar_df['Earnings Average']
        revenue_disperson = 100.0 * (calendar_df['Revenue High'] - calendar_df['Revenue Low']) / calendar_df['Revenue Average']
        calendar_result_dict[security] = {'Earn_Date': calendar_df.index.max().date(),
                                          'Earn_Dispersion': earnings_disperson.max(),
                                          'Rev_Dispersion': revenue_disperson.max()
                                         }
    
    # Relevant info extract from tipranks earnings data
    if tipranks_earnings_df is not None:
        tipranks_result_dict[security] = {'report_date': tipranks_earnings_df.report_date.max(), 
                                          'eps_forecast': tipranks_earnings_df.eps_forecast.max(),
                                          'eps_prev_yr': tipranks_earnings_df.eps_prev_yr.max()
                                         }
    
    # Info extract from options data    
    open_interest = option_df.Open_Int.sum()
    trade_volume = option_df.Vol.sum()

    # Remove all option data for which DTE < 0 (expired options)
    option_df.insert(loc=option_df.columns.get_loc('Expiry')+1, column='dte', value=pd.to_datetime(option_df.Expiry).sub(pd.to_datetime(option_df.Quote_Time)).dt.days)
    option_df = option_df[option_df.dte >= 0]

    # Nearest expiration data
#    print(security) #debug

    nearest_expiry = option_df.Expiry.min()
    nearest_expiry_df = option_df[option_df.Expiry == nearest_expiry]
    
    # ATM Strike
    # Find row with lowest absolute difference between Strike and Underlying Price
    idx = nearest_expiry_df['Strike'].sub(nearest_expiry_df['Underlying_Price']).abs().idxmin()
    atm_strike = nearest_expiry_df.loc[idx]['Strike']
    stock_price = nearest_expiry_df.loc[idx]['Underlying_Price']


    # Nearest Expiry (ne) ATM IV
    ne_atm_df = nearest_expiry_df[nearest_expiry_df.Strike == atm_strike]


    # ATM IV for nearest expiration for all securities of interest
    call_iv = ne_atm_df[ne_atm_df.Type == 'call'].IV.mean()
    put_iv = ne_atm_df[ne_atm_df.Type == 'put'].IV.mean()

    call_prem = (ne_atm_df[ne_atm_df.Type == 'call'].Bid.mean() + ne_atm_df[ne_atm_df.Type == 'call'].Ask.mean()) / 2.0
    put_prem = (ne_atm_df[ne_atm_df.Type == 'put'].Bid.mean() + ne_atm_df[ne_atm_df.Type == 'put'].Ask.mean()) / 2.0

    call_return = 100 * call_prem / atm_strike
    put_return = 100 * put_prem / atm_strike

    ne_atm_df = ne_atm_df.assign(PriceChange1SD = ne_atm_df.Underlying_Price * ne_atm_df.IV / np.sqrt(252))
    imp_move = ne_atm_df.PriceChange1SD.sum() * 100 / stock_price
    
    result_dict[security] = {'Open_Interest': open_interest,
                             'Trade_Volume': trade_volume,
                             'Stock_Price': stock_price,
                             'Latest_Expiry': nearest_expiry, 
                             'ATM_Strike': atm_strike,
                             'Call_IV': call_iv,
                             'Put_IV': put_iv,
                             'Call_Return': call_return,
                             'Put_Return': put_return,
                             'Imp_Move': imp_move
                            }
    
    # LOTTO PICKS
    # If dte is 0, options expiring today, select next option expiration
    lotto_option_df = option_df[option_df.dte > 0]
    lotto_option_df = lotto_option_df[lotto_option_df.dte == lotto_option_df.dte.min()]
    
    
    # Find desired %OTM lotto strike
    # call strike = stock x (1 + otm)
    # put strike = stock x (1 - otm)
    # %OTM = 10% for each day before expiration.
    otm = 0.10 * (lotto_option_df.dte.min() + 1)
    
    # Find rows with lowest absolute difference between stock price and OTM target for Call and Put
    lotto_call_idx = lotto_option_df[lotto_option_df.Type == 'call']['Strike'].sub(lotto_option_df.Underlying_Price * (1.0 + otm)).abs().idxmin()
    lotto_put_idx = lotto_option_df[lotto_option_df.Type == 'put']['Strike'].sub(lotto_option_df.Underlying_Price * (1.0 - otm)).abs().idxmin()

    if pd.isna(lotto_call_idx) == False:
        lotto_call_dict[security] = lotto_option_df.loc[lotto_call_idx].to_dict()
        
    if pd.isna(lotto_put_idx) == False:
        lotto_put_dict[security] = lotto_option_df.loc[lotto_put_idx]
    
    # CALL CALENDAR SPREADS
    # [TODO]: Volatility Difference for Volatility/Calendar Spreads
    # If dte is 0, options expiring today, select next expiration
    cs_option_df = option_df[(option_df.dte > 0)]
    cs_call_df = cs_option_df[cs_option_df.Type == 'call']
    
    # Strikes and expirations available
    call_strikes = sorted(cs_call_df.Strike)
    dte_range = sorted(cs_call_df.dte.unique())
            
    if ((len(call_strikes) > 0) & (len(dte_range) > 0)):
        # Strike closest to stock price (ATM Strike)
        call_atm_idx = call_strikes.index(min(call_strikes, key=lambda x: abs(x - stock_price)))

        # Select only few rows around ATM Strikes
        strikes_num = 4
        call_strikes_low_idx = call_atm_idx - strikes_num
        call_strikes_hi_idx = call_atm_idx + strikes_num + 1

        grouped_c = cs_call_df[(cs_call_df.Strike.isin(call_strikes[call_strikes_low_idx:call_strikes_hi_idx])) & (cs_call_df.dte.isin(dte_range))][['dte', 'IV']].set_index('dte').groupby('dte', sort=True).mean()

        dte_min = grouped_c.index.min()
        
        # -ve spread, IV dropped from earliest expiry
        # +ve spread, IV increased from earliest expiry
        if pd.isna(dte_min) == False:
            grouped_c['spread'] = 100 * (grouped_c['IV'] - grouped_c.loc[dte_min].IV)
            #grouped_c['spread'] = 100 * (grouped_c.IV - grouped_c.IV.shift(1))
            #grouped_c['spread'] = grouped_c.IV.pct_change
            
            spread_max = grouped_c.spread.min()
            
            dte_spread_max = grouped_c[grouped_c.spread == spread_max].index.values[0]
            
            expiry_short = cs_call_df[cs_call_df.dte == dte_min].Expiry.min()
            expiry_long = cs_call_df[cs_call_df.dte == dte_spread_max].Expiry.min()
            
            cs_call_dict[security] = {'spread': spread_max,
                                      'dte': dte_spread_max, 
                                      'stock_price': stock_price,
                                      'atm_strike': atm_strike,
                                      'expiry_short': expiry_short, 
                                      'expiry_long': expiry_long}
    
result_df = pd.DataFrame.from_dict(result_dict, orient='index')
calendar_call_df = pd.DataFrame.from_dict(cs_call_dict, orient='index')

TQQQ
UPRO
META
AAPL
PLTR
GPRO
C
TLT
SOXL
SQQQ
DIA
USO
ABNB
ADP
AEO
AMC
AMZN
AXP
BAC
BIIB
BTU
BX
CGC
CHPT
CI
CLF
CLOV
CMCSA
COST
CRM
DAL
DE
DIS
DOCU
DOW
DVN
GME
GOOGL
HD
IBM
JNJ
KR
LMT
M
MRK
MSFT
MU
NFLX
NIO
NKE
NVDA
OKTA
PANW
PEP
PFE
PINS
PYPL
RAD
RBLX
RKT
SE
SNAP
SNOW
SOFI
TSLA
TSM
UAL
UNH
UPS
VZ
WFC
WMT
XOM
ZM
^SPX
^VIX
SPY
QQQ
IWM
LQD
UVXY
VXX
XBI
XLK
XLY
XLF
XLE
XOP
ARKK
TBT
ARKG
SMH


In [8]:
# Skip securities with low price or low open interest
cond_low_price = result_df.Stock_Price >= stock_price_threshold
cond_low_oi = result_df.Open_Interest >= open_interest_threshold 

# Securities with Low Price or Low Open Interest
low_df = result_df[~(cond_low_price & cond_low_oi)]
print('Consolidated Low Price/OI {} Securities: {}'.format(len(low_df.index), sorted(low_df.index.to_list())))
display(low_df.sort_values(['Latest_Expiry', 'Imp_Move', 'Open_Interest', 'Stock_Price'], ascending=False).style.format(formatter={'Open_Interest': '{:,.0f}', 'Trade_Volume': '{:,.0f}', 'dC_otm':'{:,.0f}', 'dP_otm':'{:,.0f}'}, precision=2))

Consolidated Low Price/OI 86 Securities: ['AAPL', 'ABNB', 'ADP', 'AEO', 'AMC', 'ARKG', 'ARKK', 'AXP', 'BAC', 'BIIB', 'BTU', 'BX', 'C', 'CGC', 'CHPT', 'CI', 'CLF', 'CLOV', 'CMCSA', 'COST', 'CRM', 'DAL', 'DE', 'DIA', 'DIS', 'DOCU', 'DOW', 'DVN', 'GME', 'GOOGL', 'GPRO', 'HD', 'IBM', 'IWM', 'JNJ', 'KR', 'LMT', 'LQD', 'M', 'META', 'MRK', 'MSFT', 'MU', 'NFLX', 'NIO', 'NKE', 'NVDA', 'OKTA', 'PANW', 'PEP', 'PFE', 'PINS', 'PLTR', 'PYPL', 'QQQ', 'RAD', 'RBLX', 'RKT', 'SE', 'SMH', 'SNAP', 'SNOW', 'SOFI', 'SOXL', 'SPY', 'SQQQ', 'TBT', 'TLT', 'TQQQ', 'TSM', 'UAL', 'UNH', 'UPRO', 'UPS', 'USO', 'UVXY', 'VXX', 'VZ', 'WMT', 'XBI', 'XLE', 'XLF', 'XLK', 'XLY', 'XOP', '^VIX']


,Open_Interest,Trade_Volume,Stock_Price,Latest_Expiry,ATM_Strike,Call_IV,Put_IV,Call_Return,Put_Return,Imp_Move
WMT,"72,482","2,777",144.24,2023-03-17,145.00,0.35,1.24,3.14,12.62,10.01
^VIX,"5,984","378,056",22.29,2023-03-01,22.00,0.89,0.76,5.61,3.59,10.37
CLOV,279,"3,443",1.11,2023-02-24,1.00,0.00,0.50,0.00,0.00,3.15
AMC,"64,902","903,424",6.26,2023-02-24,6.50,0.25,0.00,0.00,0.00,1.57
CGC,"1,390","51,647",2.39,2023-02-24,2.50,0.25,0.00,0.00,0.00,1.57
NIO,180,"95,068",10.18,2023-02-24,10.00,0.00,0.13,0.00,0.00,0.79
RAD,"4,019","25,879",4.11,2023-02-24,4.00,0.00,0.13,0.00,0.00,0.79
GPRO,"2,737","4,221",5.31,2023-02-24,5.50,0.13,0.00,0.00,0.00,0.79
PLTR,"5,072","158,778",8.36,2023-02-24,8.50,0.06,0.00,0.00,0.00,0.39
VXX,"4,283","58,734",12.22,2023-02-24,12.00,0.00,0.06,0.00,0.00,0.39


In [9]:
flds = ['Put_IV', 'Call_IV', 'Put_Return', 'Call_Return', 'Imp_Move']

for fld in flds:
    # Exclude low OI/low Price securities
    sorted_df = result_df[cond_low_price & cond_low_oi].sort_values(fld, ascending=False)

    security_set.update(set(sorted_df.index[:count]))
    security_low_set.update(set(sorted_df.index[-count:]))

cols_incl = ['Open_Interest', 'Trade_Volume', 'Stock_Price', 'Latest_Expiry', 'ATM_Strike', 'Call_IV', 'Put_IV', 'Call_Return', 'Put_Return', 'Imp_Move']
print('\nConsolidated Sell Option {} Securities: {}'.format(len(security_set), security_set))
display(sorted_df[sorted_df.index.isin(security_set)][cols_incl].sort_values(['Latest_Expiry', 'Imp_Move'], ascending=[True, False]).style.format(formatter={'Open_Interest': '{:,.0f}', 'Trade_Volume': '{:,.0f}', 'Stock_Price': '{:,.2f}', 'ATM_Strike': '{:,.2f}'}, precision=2))

print('\nConsolidated Buy Option {} Securities: {}'.format(len(security_low_set), security_low_set))
display(sorted_df[sorted_df.index.isin(security_low_set)][cols_incl].sort_values(['Latest_Expiry', 'Imp_Move'], ascending=[True, False]).style.format(formatter={'Open_Interest': '{:,.0f}', 'Trade_Volume': '{:,.0f}', 'Stock_Price': '{:,.2f}', 'ATM_Strike': '{:,.2f}'}, precision=2))


Consolidated Sell Option 6 Securities: {'XOM', 'WFC', '^SPX', 'AMZN', 'TSLA', 'ZM'}


,Open_Interest,Trade_Volume,Stock_Price,Latest_Expiry,ATM_Strike,Call_IV,Put_IV,Call_Return,Put_Return,Imp_Move
^SPX,"228,922","961,048","3,991.05",2023-02-23,"3,990.00",0.33,0.10,0.69,0.18,2.71
TSLA,"826,648","2,767,672",200.86,2023-02-24,200.00,0.00,0.02,0.00,0.00,0.10
AMZN,"133,217","890,456",95.79,2023-02-24,96.00,0.01,0.00,0.00,0.00,0.05
WFC,"523,816","14,962",46.01,2023-02-24,46.00,0.00,0.00,0.00,0.00,0.01
ZM,"136,417","1,308",73.39,2023-06-16,75.00,3.15,0.53,60.40,12.83,23.21
XOM,"157,462","2,731",109.73,2023-06-16,110.00,0.24,0.87,5.00,19.30,6.97



Consolidated Buy Option 6 Securities: {'XOM', 'WFC', '^SPX', 'AMZN', 'TSLA', 'ZM'}


,Open_Interest,Trade_Volume,Stock_Price,Latest_Expiry,ATM_Strike,Call_IV,Put_IV,Call_Return,Put_Return,Imp_Move
^SPX,"228,922","961,048","3,991.05",2023-02-23,"3,990.00",0.33,0.10,0.69,0.18,2.71
TSLA,"826,648","2,767,672",200.86,2023-02-24,200.00,0.00,0.02,0.00,0.00,0.10
AMZN,"133,217","890,456",95.79,2023-02-24,96.00,0.01,0.00,0.00,0.00,0.05
WFC,"523,816","14,962",46.01,2023-02-24,46.00,0.00,0.00,0.00,0.00,0.01
ZM,"136,417","1,308",73.39,2023-06-16,75.00,3.15,0.53,60.40,12.83,23.21
XOM,"157,462","2,731",109.73,2023-06-16,110.00,0.24,0.87,5.00,19.30,6.97


In [10]:
if len(calendar_result_dict) > 0:
    calendar_result_df = pd.DataFrame.from_dict(calendar_result_dict, orient='index')
    calendar_df = calendar_result_df[calendar_result_df.Earn_Date >= pd.Timestamp.today().date()]
else:
    calendar_df = None

if len(tipranks_result_dict) > 0:
    tipranks_result_df = pd.DataFrame.from_dict(tipranks_result_dict, orient='index')
    # NOTE: the today's date is skipped if strftime is not used after today()
    cond1 = pd.to_datetime(tipranks_result_df.report_date) >= pd.Timestamp.today().strftime('%Y-%m-%d')
    cond2 = pd.to_datetime(tipranks_result_df.report_date) <= (pd.Timestamp.today() + pd.DateOffset(days=14))
    tipranks_df = tipranks_result_df[cond1 & cond2]
else:
    tipranks_df = None

if (calendar_df is not None) and (tipranks_df is not None):
    temp_df = tipranks_df.join(calendar_df, how='left')
    print('\nEarnings Calendar and Estimates for {} securities: {}'.format(len(temp_df.index), temp_df.index.to_list()))
    display(temp_df.sort_values(['report_date', 'Earn_Dispersion', 'Rev_Dispersion', 'eps_forecast'], ascending=[True, False, False, False]).style.format(precision=2))


Earnings Calendar and Estimates for 15 securities: ['ABNB', 'AEO', 'AMC', 'CHPT', 'CLOV', 'COST', 'CRM', 'KR', 'OKTA', 'PANW', 'RKT', 'SE', 'SNOW', 'SOFI', 'ZM']


,report_date,eps_forecast,eps_prev_yr,Earn_Date,Earn_Dispersion,Rev_Dispersion
RKT,2023-02-23,-0.11,0.32,2023-02-27,-333.33,28.29
ABNB,2023-02-23,0.24,0.08,nan,nan,nan
ZM,2023-02-27,0.81,1.29,2023-03-03,14.29,4.54
CLOV,2023-02-27,-0.24,-0.34,2023-02-27,-87.50,12.57
PANW,2023-02-28,0.78,0.58,2023-02-24,26.09,4.83
CRM,2023-02-28,1.36,0.84,2023-03-03,9.02,3.09
SE,2023-02-28,-0.71,-0.88,2023-03-03,-49.56,6.74
SOFI,2023-02-28,-0.09,-0.15,nan,nan,nan
SNOW,2023-03-01,0.05,0.12,2023-03-06,300.00,7.43
AEO,2023-03-01,0.26,0.35,2023-03-06,39.13,8.83


In [11]:
# Call Calendar Spread
iv_drop_threshold = -25
temp_df = calendar_call_df[calendar_call_df.spread <= iv_drop_threshold].sort_values(['dte', 'spread'], ascending=[True, True])
print('Call Calendar Spreads (IV drop by >{} points) for {} securities: {}'.format(abs(iv_drop_threshold), len(temp_df), temp_df.index.to_list()))
display(temp_df)

Call Calendar Spreads (IV drop by >25 points) for 1 securities: ['ZM']


,spread,dte,stock_price,atm_strike,expiry_short,expiry_long
ZM,-100.082855,330,73.39,75.0,2023-06-16,2024-01-19


In [12]:
print('Lotto Put Strikes (Top 10 by IV)\n')

lotto_put_df = pd.DataFrame.from_dict(lotto_put_dict, orient='index')
lotto_put_df.index.name = 'ticker'

# add %OTM column
lotto_put_df.insert(15, 'otm_pct', 100. * abs(lotto_put_df.Strike - lotto_put_df.Underlying_Price) / lotto_put_df.Underlying_Price)

# exclude low price and earnings due stocks
excl_cols = ['JSON', 'Quote_Time', 'Underlying', 'IsNonstandard', 'Root', 'PctChg', 'Chg', 'Symbol']

# exclude low price and earnings due stocks
# [NOTE]: Not correct. It excludes far away earnings too from lotto
#lotto_put_df = lotto_put_df[(lotto_put_df.Underlying_Price >= 3.0 * stock_price_threshold) & ~(lotto_put_df.index.isin(tipranks_df.index))]
lotto_put_df = lotto_put_df[(lotto_put_df.Underlying_Price >= 3.0 * stock_price_threshold)]

# exclude un-necessary columns
lotto_put_df = lotto_put_df.loc[:,~lotto_put_df.columns.isin(excl_cols)]

# Trade date within last 4 days to account for weekend and long holidays.
display(lotto_put_df[pd.to_datetime(lotto_put_df.Last_Trade_Date) >= (pd.Timestamp.now().normalize() - pd.Timedelta(4, 'd'))].sort_values(['otm_pct', 'IV'], ascending=[False, False]).head(10).style.format(precision=2, formatter={'Strike':'{:,.1f}', 'Vol':'{:,.0f}', 'Open_Int':'{:,.0f}'}))

print('Lotto Put Strikes for Holdings')
display(lotto_put_df[lotto_put_df.index.isin(etf_holdings + stock_holdings)].sort_values(['otm_pct', 'IV'], ascending=[False, False]).style.format(precision=2, formatter={'Strike':'{:,.1f}', 'Vol':'{:,.0f}', 'Open_Int':'{:,.0f}'}))

Lotto Put Strikes (Top 10 by IV)



,Strike,Expiry,dte,Type,Last,Bid,Ask,Vol,Open_Int,IV,otm_pct,Underlying_Price,Last_Trade_Date
ticker,,,,,,,,,,,,,
BAC,27.0,2023-02-24,1,put,0.01,0.00,0.00,6,0,0.50,21.26,34.29,2023-02-21 20:48:21
HD,235.0,2023-02-24,1,put,0.01,0.00,0.00,2,0,0.50,20.69,296.30,2023-02-21 19:20:32
ARKK,32.0,2023-02-24,1,put,0.02,0.00,0.00,2,0,0.50,20.54,40.27,2023-02-21 19:02:05
PYPL,60.0,2023-02-24,1,put,0.01,0.00,0.00,1,0,0.50,20.51,75.48,2023-02-21 15:37:48
NVDA,165.0,2023-02-24,1,put,0.12,0.00,0.00,"3,841",0,0.50,20.50,207.54,2023-02-22 20:59:42
MSFT,200.0,2023-02-24,1,put,0.01,0.00,0.00,22,0,0.50,20.48,251.51,2023-02-22 20:28:37
TSLA,160.0,2023-02-24,1,put,0.04,0.00,0.00,"1,732",0,0.50,20.34,200.86,2023-02-22 20:59:35
SQQQ,31.0,2023-02-24,1,put,0.01,0.00,0.00,79,0,0.50,20.19,38.84,2023-02-22 19:54:29
QQQ,235.0,2023-02-24,1,put,0.01,0.00,0.00,1,0,0.50,20.14,294.25,2023-02-22 18:25:54


Lotto Put Strikes for Holdings


,Strike,Expiry,dte,Type,Last,Bid,Ask,Vol,Open_Int,IV,otm_pct,Underlying_Price,Last_Trade_Date
ticker,,,,,,,,,,,,,
AAPL,119.0,2023-02-24,1,put,0.01,0.00,0.00,3,0,0.50,20.09,148.91,2023-02-21 19:57:31
C,40.0,2023-02-24,1,put,0.01,0.00,0.00,1,0,0.50,20.06,50.04,2023-02-22 16:33:50
META,137.0,2023-02-24,1,put,0.02,0.00,0.00,12,0,0.50,19.94,171.12,2023-02-16 20:52:52
UPRO,29.0,2023-02-24,1,put,0.05,0.00,0.00,1,0,0.50,19.60,36.07,2023-02-14 15:14:39
TQQQ,18.0,2023-02-24,1,put,0.01,0.00,0.00,455,0,0.50,19.54,22.37,2023-02-22 20:12:00


In [13]:
print('Lotto Call Strikes (Top 10 by IV)\n')

lotto_call_df = pd.DataFrame.from_dict(lotto_call_dict, orient='index')
lotto_call_df.index.name = 'ticker'

# add %OTM column
lotto_call_df.insert(15, 'otm_pct', 100. * abs(lotto_call_df.Strike - lotto_call_df.Underlying_Price) / lotto_call_df.Underlying_Price)

# exclude low price and earnings due stocks
# [NOTE]: Not correct. It excludes far away earnings too from lotto
#lotto_call_df = lotto_call_df[(lotto_call_df.Underlying_Price >= 3.0 * stock_price_threshold) & ~(lotto_call_df.index.isin(tipranks_df.index))]
lotto_call_df = lotto_call_df[(lotto_call_df.Underlying_Price >= 3.0 * stock_price_threshold)]

# exclude un-necessary columns
lotto_call_df = lotto_call_df.loc[:,~lotto_call_df.columns.isin(excl_cols)]

display(lotto_call_df[pd.to_datetime(lotto_call_df.Last_Trade_Date) >= (pd.Timestamp.now().normalize() - pd.Timedelta(4, 'd'))].sort_values(['otm_pct', 'IV'], ascending=[False, False]).head(10).style.format(precision=2, formatter={'Strike':'{:,.1f}', 'Vol':'{:,.0f}', 'Open_Int':'{:,.0f}'}))

print('Lotto Call Strikes for Holdings')
display(lotto_call_df[lotto_call_df.index.isin(etf_holdings + stock_holdings)].sort_values(['otm_pct', 'IV'], ascending=[False, False]).style.format(precision=2, formatter={'Strike':'{:,.1f}', 'Vol':'{:,.0f}', 'Open_Int':'{:,.0f}'}))

Lotto Call Strikes (Top 10 by IV)



,Strike,Expiry,dte,Type,Last,Bid,Ask,Vol,Open_Int,IV,otm_pct,Underlying_Price,Last_Trade_Date
ticker,,,,,,,,,,,,,
^VIX,38.0,2023-03-01,6,call,0.05,0.00,0.10,400,0,1.84,70.48,22.29,2023-02-22 19:21:40
LMT,590.0,2023-02-24,1,call,0.05,0.00,0.00,2,0,0.50,23.04,479.53,2023-02-21 20:35:55
DOCU,73.0,2023-02-24,1,call,0.02,0.00,0.00,76,0,0.50,20.74,60.46,2023-02-21 20:42:43
TQQQ,27.0,2023-02-24,1,call,0.01,0.00,0.00,556,0,0.50,20.70,22.37,2023-02-22 20:57:10
UNH,590.0,2023-02-24,1,call,0.01,0.00,0.00,10,0,0.50,20.68,488.89,2023-02-22 14:37:49
QQQ,355.0,2023-02-24,1,call,0.01,0.00,0.00,5,0,0.50,20.65,294.25,2023-02-21 18:59:20
XOP,155.0,2023-02-24,1,call,0.03,0.00,0.00,3,0,0.50,20.59,128.53,2023-02-22 14:42:09
PYPL,91.0,2023-02-24,1,call,0.01,0.00,0.00,8,0,0.50,20.56,75.48,2023-02-22 16:44:58
NVDA,250.0,2023-02-24,1,call,0.10,0.00,0.00,"6,113",0,0.50,20.46,207.54,2023-02-22 20:59:58


Lotto Call Strikes for Holdings


,Strike,Expiry,dte,Type,Last,Bid,Ask,Vol,Open_Int,IV,otm_pct,Underlying_Price,Last_Trade_Date
ticker,,,,,,,,,,,,,
TQQQ,27.0,2023-02-24,1,call,0.01,0.00,0.00,556,0,0.50,20.70,22.37,2023-02-22 20:57:10
C,60.0,2023-02-24,1,call,0.01,0.00,0.00,5,0,0.50,19.90,50.04,2023-02-13 15:17:32
META,205.0,2023-02-24,1,call,0.01,0.00,0.00,12,0,0.50,19.80,171.12,2023-02-22 18:26:46
UPRO,43.0,2023-02-24,1,call,0.03,0.00,0.00,2,0,0.50,19.21,36.07,2023-02-21 17:36:23
AAPL,177.5,2023-02-24,1,call,0.01,0.00,0.00,7,0,0.50,19.20,148.91,2023-02-21 18:30:10


In [14]:
print('\nStocks with potential download data error')
threshold = 0.01
for fld in ['Put_IV', 'Call_IV']:
    print('\tStocks with {} < {}'.format(fld, threshold))
    display(result_df[result_df[fld] < threshold].style.format(formatter={'Open_Interest':'{:,.0f}', 'Trade_Volume':'{:,.0f}'}, precision=2))


Stocks with potential download data error
	Stocks with Put_IV < 0.01


,Open_Interest,Trade_Volume,Stock_Price,Latest_Expiry,ATM_Strike,Call_IV,Put_IV,Call_Return,Put_Return,Imp_Move
TQQQ,"2,808","292,756",22.37,2023-02-24,22.50,0.03,0.00,0.00,0.00,0.20
AAPL,"5,678","841,789",148.91,2023-02-24,149.00,0.00,0.00,0.00,0.00,0.02
PLTR,"5,072","158,778",8.36,2023-02-24,8.50,0.06,0.00,0.00,0.00,0.39
GPRO,"2,737","4,221",5.31,2023-02-24,5.50,0.13,0.00,0.00,0.00,0.79
C,"1,495","64,583",50.04,2023-02-24,50.00,0.00,0.00,0.00,0.00,0.02
TLT,"26,435","189,706",101.31,2023-02-24,101.50,0.01,0.00,0.00,0.00,0.05
SOXL,631,"104,064",13.83,2023-02-24,14.00,0.06,0.00,0.00,0.00,0.39
SQQQ,"1,635","193,961",38.84,2023-02-24,39.00,0.02,0.00,0.00,0.00,0.10
DIA,"27,964","67,998",330.52,2023-02-24,331.00,0.01,0.00,0.00,0.00,0.05
USO,"4,184","49,363",64.92,2023-02-24,65.00,0.01,0.00,0.00,0.00,0.05


	Stocks with Call_IV < 0.01


,Open_Interest,Trade_Volume,Stock_Price,Latest_Expiry,ATM_Strike,Call_IV,Put_IV,Call_Return,Put_Return,Imp_Move
UPRO,"3,733","12,355",36.07,2023-02-24,36.00,0.00,0.02,0.00,0.00,0.10
META,"10,500","395,142",171.12,2023-02-24,170.00,0.00,0.03,0.00,0.00,0.20
AAPL,"5,678","841,789",148.91,2023-02-24,149.00,0.00,0.00,0.00,0.00,0.02
C,"1,495","64,583",50.04,2023-02-24,50.00,0.00,0.00,0.00,0.00,0.02
TLT,"26,435","189,706",101.31,2023-02-24,101.50,0.01,0.00,0.00,0.00,0.05
DIA,"27,964","67,998",330.52,2023-02-24,331.00,0.01,0.00,0.00,0.00,0.05
USO,"4,184","49,363",64.92,2023-02-24,65.00,0.01,0.00,0.00,0.00,0.05
ABNB,97,"55,222",127.21,2023-02-24,127.00,0.00,0.01,0.00,0.00,0.05
ADP,"1,301","2,631",222.93,2023-02-24,222.50,0.00,0.01,0.00,0.00,0.05
AMZN,"133,217","890,456",95.79,2023-02-24,96.00,0.01,0.00,0.00,0.00,0.05


In [15]:
print('\nNearest Expiration Dates\n\tCheck stocks with nearest weekly or monthly Friday not included')
display(result_df.sort_values(['Latest_Expiry', 'Imp_Move', 'Open_Interest', 'Stock_Price'], ascending=False)[['Latest_Expiry', 'Open_Interest', 'Stock_Price', 'Imp_Move']].style.format(formatter={'Open_Interest': '{:,.0f}'}, precision=2))


Nearest Expiration Dates
	Check stocks with nearest weekly or monthly Friday not included


,Latest_Expiry,Open_Interest,Stock_Price,Imp_Move
ZM,2023-06-16,"136,417",73.39,23.21
XOM,2023-06-16,"157,462",109.73,6.97
WMT,2023-03-17,"72,482",144.24,10.01
^VIX,2023-03-01,"5,984",22.29,10.37
CLOV,2023-02-24,279,1.11,3.15
AMC,2023-02-24,"64,902",6.26,1.57
CGC,2023-02-24,"1,390",2.39,1.57
NIO,2023-02-24,180,10.18,0.79
RAD,2023-02-24,"4,019",4.11,0.79
GPRO,2023-02-24,"2,737",5.31,0.79
